In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm_notebook

%matplotlib inline

PATH = '../'

In [57]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.54 s


### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [58]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

### Model training

In [59]:
submTrain = pd.read_csv('submTrain.csv').iloc[:, 1:]
dataSubm = pd.read_csv('dataSubm.csv')

In [62]:
train = train.iloc[submTrain.index, :]

In [68]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [69]:
X = pd.concat([X, submTrain.iloc[:, 2:]], axis = 1)

In [71]:
X.drop(columns = 'epoch', inplace = True)
Xtest.drop(columns = 'epoch', inplace = True)

In [72]:
X['r'] = np.sqrt(X.x_sim**2 + X.y_sim**2 + X.z_sim**2)
Xtest['r'] = np.sqrt(Xtest.x_sim**2 + Xtest.y_sim**2 + Xtest.z_sim**2)

X['phi'] = np.arctan(X.y_sim/X.x_sim)
Xtest['phi'] = np.arctan(Xtest.y_sim/Xtest.x_sim)

X['theta'] = np.arccos(X.z_sim/X.r)
Xtest['theta'] = np.arccos(Xtest.z_sim/Xtest.r)



X['Vr'] = np.sqrt(X.Vx_sim**2 + X.Vy_sim**2 + X.Vz_sim**2)
Xtest['Vr'] = np.sqrt(Xtest.Vx_sim**2 + Xtest.Vy_sim**2 + Xtest.Vz_sim**2)

X['Vphi'] = np.arctan(X.Vy_sim/X.Vx_sim)
Xtest['Vphi'] = np.arctan(Xtest.Vy_sim/Xtest.Vx_sim)

X['Vtheta'] = np.arccos(X.Vz_sim/X.Vr)
Xtest['Vtheta'] = np.arccos(Xtest.Vz_sim/Xtest.Vr)

In [78]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
model = Ridge(alpha=0.01)
trainSize = 0.8
submissionValid = train.copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
smp = []
SMP = []
nTargets = 6
for sat_id in tqdm_notebook(Xtest['sat_id'].unique()):
    if sat_id >= 0:
        smp = []
        satX = X[X['sat_id'] == sat_id]
#         satX = pd.concat([satX, satX.shift(1).fillna(0), 
#                           satX.shift(2).fillna(0), satX.shift(3).fillna(0),
#                          satX.shift(4).fillna(0)], axis = 1)
        satY = y[X['sat_id'] == sat_id]

        size = int(satX.shape[0] * trainSize)
        Xtr, Xval = satX.iloc[:size, :], satX.iloc[size:, :]
        ytr, yval = satY.iloc[:size, :], satY.iloc[size:, :]
        for i in range(nTargets):
            model.fit(Xtr, ytr.iloc[:, i])
            ypred = model.predict(Xval)
            smp.append(smape(ypred, yval.iloc[:, i]))
            SMP.append(smape(ypred, yval.iloc[:, i]))
print('Final', 100*(1- np.mean(SMP)))


Final 83.1306366174423


In [8]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor
model = Ridge(alpha=0.01)
nTargets = 6
for sat_id in tqdm_notebook(Xtest['sat_id'].unique()):
    satX = X[X['sat_id'] == sat_id].drop(columns = ['sat_id'])
    satX = pd.concat([satX, satX.shift(1).fillna(0), 
                          satX.shift(2).fillna(0), satX.shift(3).fillna(0),
                         satX.shift(4).fillna(0)], axis = 1)
    satY = y[X['sat_id'] == sat_id]
    satXtest = Xtest[Xtest['sat_id'] == sat_id].drop(columns = ['sat_id'])
    satXtest = pd.concat([satXtest, satXtest.shift(1).fillna(0),satXtest.shift(2).fillna(0), satXtest.shift(3).fillna(0),
                         satXtest.shift(4).fillna(0)], axis = 1)
    
    for i in range(nTargets):
        model.fit(satX, satY.iloc[:, i])
        ypred = model.predict(satXtest)
        submission.loc[satXtest.index, submission.columns[i+1]] = ypred

In [11]:
submission.to_csv('submission.csv', index = None)

NameError: name 'submission' is not defined

In [12]:
import pandas as pd
train = pd.read_csv('simulationTest.csv')

In [15]:
test = pd.read_csv('../' + 'Track 1/test.csv')

In [17]:
test.head(10)

,id,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,3927,1,2014-02-01T00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,3928,1,2014-02-01T00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,3929,1,2014-02-01T00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,3930,1,2014-02-01T01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,3931,1,2014-02-01T01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724
5,3932,1,2014-02-01T01:47:44.386,10811.062601,5601.275788,-1958.108672,-1.703730,5.680489,-3.254421
6,3933,1,2014-02-01T02:08:56.231,7207.568370,11719.756441,-5639.974363,-3.657275,3.897901,-2.482004
7,3934,1,2014-02-01T02:30:08.076,2087.178056,15604.788815,-8261.164504,-4.242309,2.279618,-1.661818
8,3935,1,2014-02-01T02:51:19.921,-3328.823919,17694.467481,-9937.160206,-4.211431,1.069290,-1.001519
9,3936,1,2014-02-01T03:12:31.766,-8521.041748,18460.550588,-10872.042252,-3.927017,0.180062,-0.490210


In [18]:
train.head(10)

,id,x,y,z,Vx,Vy,Vz
0,3927,-24793.184454,-10909.866397,6569.504253,2.614113,-1.304336,1.087089
1,3928,-21091.733626,-12383.587916,7838.189239,3.218799,-0.995557,0.895872
2,3929,-16588.072813,-13379.650876,8803.865646,3.877186,-0.541540,0.602224
3,3930,-11215.499140,-13659.244683,9296.935345,4.581871,0.153877,0.135990
4,3931,-4952.335469,-12805.666122,9016.311158,5.250625,1.287673,-0.650191
5,3932,1945.165461,-10037.336267,7381.103277,5.434240,3.248744,-2.062864
6,3933,7862.968947,-4114.880553,3404.622192,3.170376,6.082549,-4.241243
7,3934,8307.228647,4162.285412,-2574.726439,-2.474540,6.082846,-4.543688
8,3935,3178.259144,10169.985040,-7204.448566,-4.926661,3.383852,-2.715480
9,3936,-3237.336461,13161.226282,-9713.858324,-4.990148,1.488338,-1.343726


In [6]:
train.loc[:, 'x'] = 0

In [8]:
train.to_csv('hmm.csv', index = None)

In [10]:
train

,id,x,y,z,Vx,Vy,Vz
0,3927,0,-10909.866397,6569.504253,2.614113,-1.304336,1.087089
1,3928,0,-12383.587916,7838.189239,3.218799,-0.995557,0.895872
2,3929,0,-13379.650876,8803.865646,3.877186,-0.541540,0.602224
3,3930,0,-13659.244683,9296.935345,4.581871,0.153877,0.135990
4,3931,0,-12805.666122,9016.311158,5.250625,1.287673,-0.650191
5,3932,0,-10037.336267,7381.103277,5.434240,3.248744,-2.062864
6,3933,0,-4114.880553,3404.622192,3.170376,6.082549,-4.241243
7,3934,0,4162.285412,-2574.726439,-2.474540,6.082846,-4.543688
8,3935,0,10169.985040,-7204.448566,-4.926661,3.383852,-2.715480
9,3936,0,13161.226282,-9713.858324,-4.990148,1.488338,-1.343726
